In [ ]:
import VASPsol as vs
from dask.distributed import Client, LocalCluster, progress
import os
import shutil as sh
import numpy as np
import time
import dask

def status(jobs):
    st = ['pending','finished','error']
    status = []
    finished_jobs = []
    for s in st:
        count = 0
        for job in jobs:
           # count = 0
            if job.status == s:
                count += 1
                if s == 'finished':
                    finished_jobs.append(job)
        status.append(count)
    return status, finished_jobs

def solv_calc(file, nc, sig, tau):
    # This is where you can change the solvent that is being run~!!!
    return vs.calculate_solvent(file, NSW=0, solvent='water', dict_of_additional_tags={'NC_K':nc, 'SIGMA_K':sig, 'TAU':tau})

def vac_calc(file):
    return vs.calculate_vacuum(file, NSW=0)


In [2]:

n_jobs = 4
client, cluster = vs.ef.dask_workers(16, 4, 1, burst=False)
cluster.scale(n_jobs)
client.wait_for_workers(n_jobs)
#cluster = LocalCluster(n_workers=1)
#client = Client(cluster)
print('ALL WORKERS PRESENT')

directories = [os.path.abspath(i) for i in next(os.walk('.'))[1] if 'POSCAR' in next(os.walk(i))[2]]

### FIRST RUN VACUUM CALCUALTIONS


#vac_jobs = client.map(vac_calc, directories)

### THEN RUN 

nc_k = np.linspace(1e-3, 4e-3, 4)
sigma = np.linspace(0.2, 1.2, 4)
tau = np.linspace(1e-5, 1e-3, 4)





/home/ericfonseca/.local/lib/python3.10/site-packages/dask_jobqueue-0.8.1-py3.10.egg/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/ericfonseca/.local/lib/python3.10/site-packages/dask_jobqueue-0.8.1-py3.10.egg/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/ericfonsec

ALL WORKERS PRESENT


In [3]:
jobs = []
for nc in nc_k:
    for sig in sigma:
        for t in tau:
            for file in directories:
                jobs.append(dask.delayed(solv_calc)(file, nc, sig, t))


In [4]:
# shuffle the order of the jobs
np.random.shuffle(jobs)
jobs

[Delayed('solv_calc-32423966-1755-40bb-a8e4-37c9cbfa38c1'),
 Delayed('solv_calc-39a96ec0-311a-4b89-84da-861302127fb3'),
 Delayed('solv_calc-4b9c39dd-ad48-4c79-9218-c516cbfb90b6'),
 Delayed('solv_calc-a6192f7b-ee21-44cf-8225-72c4cdcb58c1'),
 Delayed('solv_calc-41a8ead9-700a-4329-8158-8f51ef11672b'),
 Delayed('solv_calc-2fea0bfe-38d1-4fd9-856f-d187e113ebf3'),
 Delayed('solv_calc-ad9e0c1d-8fbd-4891-9a19-e61f936f8fb7'),
 Delayed('solv_calc-7c6c79a0-2d35-4e53-8cce-ffdd1e158cba'),
 Delayed('solv_calc-278d8724-7896-41bc-829f-6239f4d69455'),
 Delayed('solv_calc-f152e019-1987-4daa-8a45-f2b1d935a521'),
 Delayed('solv_calc-021e3be6-aa16-4129-9a1b-cb003250480f'),
 Delayed('solv_calc-87cf7756-8d1a-4b31-bb84-d6d1786ad717'),
 Delayed('solv_calc-046d6598-110f-4a5f-9e61-e990272d74e1'),
 Delayed('solv_calc-1ecd0623-0b2f-48a7-be5e-00131e938272'),
 Delayed('solv_calc-21659940-b582-45b0-934f-03f6f00d877e'),
 Delayed('solv_calc-9fb924ea-e34f-44d2-8d07-e3c1615879e0'),
 Delayed('solv_calc-96eb7c8d-8f0f-4759-b

In [ ]:
# grab the first 10 jobs
test_jobs = 

In [ ]:
jobs = []
for nc in nc_k:
    for sig in sigma:
        for t in tau:
            for file in directories:
                jobs.append(client.submit(solv_calc, file, nc, sig, t))

counts, finished_jobs = status(jobs)
complete = counts[1]+counts[2]
while complete < len(jobs):
    counts, finished_jobs = status(jobs)
    complete = counts[1] + counts[2]
    print('PENDING : ',counts[0], ' | FINISHED : ', counts[1], ' | ERROR : ', counts[2],' ***************')
    time.sleep(10)

jobs = client.gather(jobs)